In [1]:


from productivity_bot.actions.planner_action import get_planner_system_message, PlannerAction


In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

planner_agent = AssistantAgent(
    name="planner_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o", api_key="…"),
    system_message=get_planner_system_message(),
    output_content_type=PlannerAction,
)

response = await planner_agent.on_user_message(UserMessage("reschedule", source="user"))

AttributeError: 'AssistantAgent' object has no attribute 'on_user_message'

In [ ]:
# Test the canonical intent parsing agent
async def test_canonical_agent():
    """Test the canonical intent parsing agent with various inputs."""
    
    # You'll need to set your OpenAI API key
    api_key = "your-openai-api-key-here"  # Replace with actual key
    agent = CanonicalIntentParsingAgent(api_key)
    
    # Test cases covering different intent types
    test_inputs = [
        "postpone this for 30 minutes",
        "done, all finished",
        "create a new calendar event",
        "gimme 5 minutes",
        "mark this as complete", 
        "schedule this for 8pm tomorrow",
        "delay for an hour",
        "I'm all set, wrap it up",
        "random unclear message",
        "not now, maybe later"
    ]
    
    print("Testing Canonical Intent Parsing Agent")
    print("=" * 50)
    
    for i, user_input in enumerate(test_inputs, 1):
        try:
            action = await agent.parse_intent(user_input)
            print(f"{i:2d}. Input: '{user_input}'")
            print(f"    Action: {action.action}")
            if action.minutes:
                print(f"    Minutes: {action.minutes}")
            if action.commitment_time:
                print(f"    Commitment: {action.commitment_time}")
            print()
        except Exception as e:
            print(f"{i:2d}. Input: '{user_input}' - ERROR: {e}")
    
    # Test batch processing
    print("Testing batch processing...")
    batch_results = await agent.batch_parse_intents(test_inputs[:3])
    for i, (input_text, result) in enumerate(zip(test_inputs[:3], batch_results)):
        print(f"Batch {i+1}: '{input_text}' -> {result.action}")

# Uncomment and run this to test (you'll need to add your OpenAI API key)
# await test_canonical_agent()